In [9]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import datetime
import pickle
import gzip
import matplotlib.pyplot as plt

In [10]:
tech_video_metadata = "/Users/salma/Desktop/ADA/project/datasets/tech_review_videos.p"
video_comments ="/Users/salma/Desktop/ADA/project/datasets/num_comments.tsv.gz"

In [11]:
ITEMS = ['iphone x', 'iphone 7', 'iphone 6', 'galaxy s9', 'galaxy note 9', 'galaxy fold']
RELEASE_DATES = {
    'iphone x': '2017-11-03',
    'iphone 7': '2016-09-16',
    'iphone 6': '2014-09-19',
    'galaxy s9': '2018-03-16',
    'galaxy note 9': '2018-08-24',
    'galaxy fold': '2019-02-01'
}


In [12]:
with gzip.open(tech_video_metadata, "rb") as f:
    df_video = pickle.load(f, encoding='bytes')
display(df_video)

,channel_id,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
0,UCzWQYUVCpZqtN93H8RR44Qw,337.0,_z8PiR7OkIg,235,12122.0,"seeker,science,curiosity,education,explain,lea...",WiFi 6 Just Launched and Here’s What It Means ...,2019-09-26,419824.0
1,UCzWQYUVCpZqtN93H8RR44Qw,186.0,Yv8yw4EgMYU,799,5451.0,"battery,batteries,charge,life,long,last,chargi...","Batteries Come in All Shapes & Sizes, But Each...",2018-04-05,163137.0
2,UCzWQYUVCpZqtN93H8RR44Qw,365.0,HF-uEK0fDgQ,261,3956.0,"current events,Science,video,Hacking,security,...",Hackers Can Now Break Into Your Phone Using Music,2017-04-25,117980.0
3,UCzWQYUVCpZqtN93H8RR44Qw,61.0,YMzN_a66NKg,217,3390.0,"current events,Science,iPhone,elements,rare,sc...",Why It Takes 75 Elements To Make Your Cell Phone,2016-11-26,126637.0
4,UCzWQYUVCpZqtN93H8RR44Qw,252.0,TkEMPh0cXUw,293,7853.0,"current events,Science,battery life,smartphone...",Why Does Your Phone Battery Suck?,2016-10-02,283795.0
...,...,...,...,...,...,...,...,...,...
85940,UCrxStpzUb40yGqo_EaI972g,0.0,my-xzYvHtsk,131,1.0,"Huawei P20 Pro,DSLR KILLER,World's First Tripp...",Huawei P20 Pro: DSLR KILLER? World's First Tri...,2018-03-28,78.0
85941,UCrxStpzUb40yGqo_EaI972g,6.0,j00Q2AFlxCM,135,34.0,"Samsung Galaxy J3 Prime 2018,Specifications,Fe...",Samsung Galaxy J3 Prime (2018) Full Phone Spec...,2018-03-20,8421.0
85942,UCrxStpzUb40yGqo_EaI972g,4.0,sc9PEm11cQg,134,6.0,"iphone xi 2018,iphone concept,iphone,concept,i...",iPhone XI 2018 Concept - World First 5G Phone ...,2018-03-16,858.0
85943,UCrwPkTBuhZJDgiHkWzY7COw,34.0,pAgbCUhkh7w,146,244.0,,how to fix unfortunately the process com.googl...,2018-03-02,28259.0


#### treat_i is equal to 1, this means that the video talks about the event. Otherwise, it does not.

In [13]:
df_video["treat_iph_x"]=0
df_video["treat_iph_7"]=0
df_video["treat_iph_6"]=0
df_video["treat_gal_S9"]=0
df_video["treat_gal_note_9"]=0
df_video["treat_iph_gal_fold"]=0



In [14]:
df_video

,channel_id,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,treat_iph_x,treat_iph_7,treat_iph_6,treat_gal_S9,treat_gal_note_9,treat_iph_gal_fold
0,UCzWQYUVCpZqtN93H8RR44Qw,337.0,_z8PiR7OkIg,235,12122.0,"seeker,science,curiosity,education,explain,lea...",WiFi 6 Just Launched and Here’s What It Means ...,2019-09-26,419824.0,0,0,0,0,0,0
1,UCzWQYUVCpZqtN93H8RR44Qw,186.0,Yv8yw4EgMYU,799,5451.0,"battery,batteries,charge,life,long,last,chargi...","Batteries Come in All Shapes & Sizes, But Each...",2018-04-05,163137.0,0,0,0,0,0,0
2,UCzWQYUVCpZqtN93H8RR44Qw,365.0,HF-uEK0fDgQ,261,3956.0,"current events,Science,video,Hacking,security,...",Hackers Can Now Break Into Your Phone Using Music,2017-04-25,117980.0,0,0,0,0,0,0
3,UCzWQYUVCpZqtN93H8RR44Qw,61.0,YMzN_a66NKg,217,3390.0,"current events,Science,iPhone,elements,rare,sc...",Why It Takes 75 Elements To Make Your Cell Phone,2016-11-26,126637.0,0,0,0,0,0,0
4,UCzWQYUVCpZqtN93H8RR44Qw,252.0,TkEMPh0cXUw,293,7853.0,"current events,Science,battery life,smartphone...",Why Does Your Phone Battery Suck?,2016-10-02,283795.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85940,UCrxStpzUb40yGqo_EaI972g,0.0,my-xzYvHtsk,131,1.0,"Huawei P20 Pro,DSLR KILLER,World's First Tripp...",Huawei P20 Pro: DSLR KILLER? World's First Tri...,2018-03-28,78.0,0,0,0,0,0,0
85941,UCrxStpzUb40yGqo_EaI972g,6.0,j00Q2AFlxCM,135,34.0,"Samsung Galaxy J3 Prime 2018,Specifications,Fe...",Samsung Galaxy J3 Prime (2018) Full Phone Spec...,2018-03-20,8421.0,0,0,0,0,0,0
85942,UCrxStpzUb40yGqo_EaI972g,4.0,sc9PEm11cQg,134,6.0,"iphone xi 2018,iphone concept,iphone,concept,i...",iPhone XI 2018 Concept - World First 5G Phone ...,2018-03-16,858.0,0,0,0,0,0,0
85943,UCrwPkTBuhZJDgiHkWzY7COw,34.0,pAgbCUhkh7w,146,244.0,,how to fix unfortunately the process com.googl...,2018-03-02,28259.0,0,0,0,0,0,0


In [22]:
print(df_video["title"])

0        WiFi 6 Just Launched and Here’s What It Means ...
1        Batteries Come in All Shapes & Sizes, But Each...
2        Hackers Can Now Break Into Your Phone Using Music
3         Why It Takes 75 Elements To Make Your Cell Phone
4                        Why Does Your Phone Battery Suck?
                               ...                        
85940    Huawei P20 Pro: DSLR KILLER? World's First Tri...
85941    Samsung Galaxy J3 Prime (2018) Full Phone Spec...
85942    iPhone XI 2018 Concept - World First 5G Phone ...
85943    how to fix unfortunately the process com.googl...
85944    Unboxing Samsung J2 (2018) super budget smartp...
Name: title, Length: 85945, dtype: object
